## time process

In [4]:
configToReadCsv ='''

dbms.security.allow_csv_import_from_file_urls=true

#server.directories.import=import


'''

deleteAll = '''

MATCH (n)-[r]-() DELETE r, n;
'''


LOAD CSV WITH HEADERS FROM 'file:////Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/Code/DataSet/JobDataRDW256MB.csv' AS row
MERGE (job:FactJobPosting {ID: toInteger(row.ID)})
ON CREATE SET job.totalJobPost = toInteger(row.jobCount)

MERGE (career:Career {name: COALESCE(row.Career, 'Unknown')})
MERGE (job)-[:Belong_to_career]->(career)

MERGE (location:Location {name: COALESCE(row.Location, 'Unknown')})
MERGE (job)-[:Located_in]->(location)

MERGE (time:Time {name: COALESCE(row.Time, 'Unknown')})
MERGE (job)-[:Posted_at_time]->(time)

FOREACH(ignoreMe IN CASE WHEN row.ProgrammingLanguage IS NOT NULL THEN [1] ELSE [] END |
    MERGE (pl:ProgrammingLanguage {programmingLanguage: row.ProgrammingLanguage})
    MERGE (job)-[:Required_programmingLanguage]->(pl)
)

FOREACH(ignoreMe IN CASE WHEN row.Framework IS NOT NULL THEN [1] ELSE [] END |
    MERGE (fw:Framework {framework: row.Framework})
    MERGE (job)-[:Required_framework]->(fw)
)

FOREACH(ignoreMe IN CASE WHEN row.Knowledge IS NOT NULL THEN [1] ELSE [] END |
    MERGE (kn:Knowledge {knowledge: row.Knowledge})
    MERGE (job)-[:Required_knowledge]->(kn)
)

FOREACH(ignoreMe IN CASE WHEN row.Tool IS NOT NULL THEN [1] ELSE [] END |
    MERGE (tl:Tool {tool: row.Tool})
    MERGE (job)-[:Required_tool]->(tl)
)

FOREACH(ignoreMe IN CASE WHEN row.Platform IS NOT NULL THEN [1] ELSE [] END |
    MERGE (pf:Platform {platform: row.Platform})
    MERGE (job)-[:Required_platform]->(pf)
)

FOREACH(ignoreMe IN CASE WHEN row.Organization IS NOT NULL THEN [1] ELSE [] END |
    MERGE (o:Organization {name: row.Organization})
    MERGE (job)-[:Recruited_by]->(o)
)

MERGE (web:Website {name: COALESCE(row.Web, 'Unknown')})
ON CREATE SET web.URL = COALESCE(row.Web, 'Unknown')
MERGE (job)-[:Published_on]->(web);




1mb:
Added 67 labels, created 67 nodes, set 69 properties, created 66 relationships, completed after 2137 ms.



LOAD CSV WITH HEADERS FROM 'file:////Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/Code/DataSet/courseDataForRDW3MB.csv' AS row
MERGE (course:Course {ID: row.ID})
ON CREATE SET course.CourseName = row.CourseName, course.Link = row.Link, course.Level = row.Level, course.Duration = toFloat(row.Duration), course.Price = toFloat(row.Price)

MERGE (factCourse:FactCourse {ID: row.ID})
ON CREATE SET factCourse.Enroll = toInteger(row.Enroll), factCourse.Rate = toFloat(row.Rate)
MERGE (factCourse)-[:Belong_to_course]->(course)

MERGE (website:Website {name: coalesce(row.Website, 'Unknown')})
ON CREATE SET website.URL = coalesce(row.LinkToWebsite, 'Unknown')
MERGE (factCourse)-[:Posted_on]->(website)

FOREACH(ignoreMe IN CASE WHEN row.ProgrammingLanguage IS NOT NULL THEN [1] ELSE [] END |
    MERGE (pl:ProgrammingLanguage {programmingLanguage: row.ProgrammingLanguage})
    MERGE (factCourse)-[:Taught_programmingLanguage]->(pl)
)

FOREACH(ignoreMe IN CASE WHEN row.Knowledge IS NOT NULL THEN [1] ELSE [] END |
    MERGE (kn:Knowledge {knowledge: row.Knowledge})
    MERGE (factCourse)-[:Taught_knowledge]->(kn)
)

FOREACH(ignoreMe IN CASE WHEN row.Platform IS NOT NULL THEN [1] ELSE [] END |
    MERGE (pf:Platform {platform: row.Platform})
    MERGE (factCourse)-[:Taught_platform]->(pf)
)

FOREACH(ignoreMe IN CASE WHEN row.Framework IS NOT NULL THEN [1] ELSE [] END |
    MERGE (fw:Framework {framework: row.Framework})
    MERGE (factCourse)-[:Taught_framework]->(fw)
)

FOREACH(ignoreMe IN CASE WHEN row.Tool IS NOT NULL THEN [1] ELSE [] END |
    MERGE (tl:Tool {tool: row.Tool})
    MERGE (factCourse)-[:Taught_tool]->(tl)
)

FOREACH(ignoreMe IN CASE WHEN row.Organization IS NOT NULL THEN [1] ELSE [] END |
    MERGE (o:Organization {name: row.Organization})
    MERGE (factCourse)-[:Belong_to]->(o)
)

MERGE (pl)-[:Have_framework]->(fw)
MERGE (pl)-[:Use_tool]->(tl)
MERGE (pl)-[:Relate_to_knowledge]->(kn)
MERGE (fw)-[:Deploy_to_platform]->(pf)




In [ ]:
q1 =f'''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)-[:Posted_on]->(w:Website) where w.name = "coursera" 
return c 

'''

q2 =f'''
match(f:FactJobPosting)-[:Belong_to_career]->(c:Career) where c.name = "Data Engineer" return sum(f.totalJobPost) as JobCount



'''

q3 =f'''

match(c:Course)<-[:Belong_to_course]-(f:FactCourse) where exists {(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]} or exists {(f)-[:Taught_framework]->(fw:Framework) where fw.framework in ["SPRING", "SPARK"]} or exists {(f)-[:Taught_tool]->(tl:Tool) where tl.tool = "GIT"} or exists {(f)-[:Taught_knowledge]->(kl:Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]} or exists {(f)-[:Taught_platform]->(pf:Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]}
return distinct c
'''

q4 =f'''

match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return c.name as CareerName,
 pl.programmingLanguage as ProgrammingLanguage,
  kl.knowledge as Knowledge,   fw.framework as Framework,      tl.tool as Tool, 
pf.platform as Platform
'''

q5 =f'''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
 match(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Taught_framework]->(fw:Framework) where fw.framework in ["SPRING", "SPARK"]
optional match(f)-[:Taught_tool]->(tl:Tool) where tl.tool = "GIT"
optional match(f)-[:Taught_knowledge]->(kl:Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Taught_platform]->(pf:Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return p, tl, kl, fw, pf, count(c) as NumberOfCourse order by NumberOfCourse desc
'''

q6 =f'''
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return c.name, pl.programmingLanguage, kl, fw.framework, tl, pf, sum(f.totalJobPost) as NumberOfJob order by NumberOfJob desc
'''

match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
optional match(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) 
optional match(f)-[:Taught_framework]->(fw:Framework)
optional match(f)-[:Taught_tool]->(tl:Tool) 
optional match(f)-[:Taught_knowledge]->(kl:Knowledge) 
optional match(f)-[:Taught_platform]->(pf:Platform) 
return distinct c, collect(distinct pl.programmingLanguage) as ProgrammingLanguage,
    collect(distinct kl.knowledge) as Knowledge,  collect(distinct fw.framework) as Framework, collect(distinct tl.tool) as Tool, 
    collect(distinct pf.platform) as Platform


In [21]:
import pandas as pd
from py2neo import Graph, Node, Relationship

DEFAULT_PATH = "/Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/Code/DataSet/"
# Initialize Spark session


# Load CSV into Spark DataFrame

# Connect to Neo4j
graph = Graph("bolt://localhost:7687", user="neo4j", password="12345678")

In [2]:

def getOrCreateNode(nodeName, attribute, attributeValue, level=None):
    
    cypher_query = ""
    if (pd.isna(attributeValue)): return None

    # check if value checker is string of not
    if (type(attributeValue) == str): 
        if(len(attributeValue) < 2): return None
        
        attributeValue = attributeValue.replace("\"", "'")

        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}=\"{attributeValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}={attributeValue} RETURN n"

    result = graph.evaluate(cypher_query)
    if (result is None):
        newNode = Node(nodeName, **{attribute: attributeValue})
        result = graph.create(newNode)
        result = newNode
    

    return result


def getOrCreatetMonthNode(month, year):
    
    cQuery = f"MATCH (m: Month {{month:{month}}})-[:part_of]->(y: Year {{year:{year}}})  RETURN m"
    result = graph.evaluate(cQuery)

    if result is None:
        Year = getOrCreateNode("Year", "year", year)
        Month = Node("Month", month=month)
        graph.create(Month)
        graph.merge(Relationship(Month, "part_of", Year))
        return Month

    return result



def getOrCreatetDateNode(date, month, year):

    # Construct the WHERE clause for the Cypher query
    cypher_query = f"MATCH (n: Date {{day:{date}}})-[:part_of]->(m: Month {{month:{month}}})-[:part_of]->(y: Year {{year:{year}}})  RETURN n"
    
    result = graph.evaluate(cypher_query)

    if result is None:
        Month = getOrCreatetMonthNode(month, year)
        Date = Node("Date", day=date)
        graph.create(Date)
        graph.merge(Relationship(Date, "part_of", Month))
        return Date

    return result


In [7]:

# Function to process each row of DataFrame
def process_row(row):
    job_node = Node("FactJobPosting", ID=int(row['ID']), totalJobPost=int(row['jobCount']))
    graph.merge(job_node, "FactJobPosting", "ID")
    
    # Career
    career_node = Node("Career", name=row['Career'])
    graph.merge(career_node, "Career", "name")
    graph.merge(Relationship(job_node, "Belong_to_career", career_node))
    
    # Location
    location_node = Node("Location", name=row['Location'] )
    graph.merge(location_node, "Location", "name")
    graph.merge(Relationship(job_node, "Located_in", location_node))

    # Time
    time_node =  getOrCreatetDateNode(row['Date'],row['Month'], row['Year'])
    graph.merge(Relationship(job_node, "Posted_at_time", time_node))

    # Conditional relationships for various properties
    for field, label, attr in [
        ("ProgrammingLanguage", "Required_programmingLanguage", "programmingLanguage"),
        ("Framework", "Required_framework", "framework"),
        ("Knowledge", "Required_knowledge", "knowledge"),
        ("Tool", "Required_tool", "tool"),
        ("Platform", "Required_platform", "platform"),
        ("Organization", "Recruited_by", "name")
    ]:
        if pd.notnull(row[field]):
            node = Node(field, **{attr:row[field]})
            graph.merge(node, field, attr)
            graph.merge(Relationship(job_node, label, node))

    # Website
    web_node = Node("Website", name=row['Web'] if pd.notnull(row['Web']) else 'Unknown')
    graph.merge(web_node, "Website", "name")
    # Assuming URL should be a property of the Website node, you might want to adjust this.
    web_node['URL'] = row['Web'] if pd.notnull(row['Web']) else 'Unknown'
    graph.push(web_node)
    graph.merge(Relationship(job_node, "Published_on", web_node))

# Iterate over each row and process it
def load2Neo4j(data: pd.DataFrame):
    l = len(data)
    for index, row in data.iterrows():
        process_row(row)
        print(f"Loading process {index/l * 100} %", end='\r')



In [9]:
data = pd.read_csv(DEFAULT_PATH + "JobDataRDW256MB.csv")
print(len(data))


2032291


In [11]:

# create a new node without checking anything
def createNewNode(nodeName, propertiesList):
    properties = {k: v for k, v in propertiesList.items()}
    new_node = Node(nodeName, **properties)
    graph.create(new_node)
    return new_node

#----------------------------------------------------------------
# create new node if it does not exist else merge that node with single property
#----------------------------------------------------------------
def getOrCreateNode(nodeName, attribute, attributeValue, level=None):
    
    cypher_query = ""
    if (pd.isna(attributeValue)): return None

    # check if value checker is string of not
    if (type(attributeValue) == str): 
        if(len(attributeValue) < 2): return None
        
        attributeValue = attributeValue.replace("\"", "'")

        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}=\"{attributeValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attribute}={attributeValue} RETURN n"

    result = graph.evaluate(cypher_query)
    if (result is None):
        newNode = Node(nodeName, **{attribute: attributeValue})
        result = graph.create(newNode)
        result = newNode
    

    return result


#----------------------------------------------------------------
# create new node if it does not exist else merge that node with multi properties
#----------------------------------------------------------------

def getOrCreateNodeMultiAttr(nodeName, attrChecker, AttrCheckerValue, propertiesList, level=None):
    # Construct the WHERE clause for the Cypher query
    if (pd.isna(AttrCheckerValue)): return None
    cypher_query =""
    if (type(AttrCheckerValue) == str): 
        if len(AttrCheckerValue) < 2:  return None
        propertiesList[attrChecker] = AttrCheckerValue.replace("\"", "'")
        AttrCheckerValue = AttrCheckerValue.replace("\"", "'")
        if level is not None:
            cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" and n.level = \"{level}\" RETURN n"
        else:
            if nodeName == "Framework" or nodeName == "ProgrammingLanguage":
                cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" and n.level is null RETURN n"
            else:
                cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}=\"{AttrCheckerValue}\" RETURN n"
    else:
        cypher_query = f"MATCH (n:{nodeName}) WHERE n.{attrChecker}={AttrCheckerValue} RETURN n"

    result = graph.evaluate(cypher_query)

    if result is None:
        # Dynamically set properties using keyword arguments
        properties = {k: v for k, v in propertiesList.items()}
        new_node = Node(nodeName, **properties)
        graph.create(new_node)
        return new_node

    return result

def makeRelationship(sourceNode, relation, targetNode):
    if (sourceNode is None or targetNode is None): return
    graph.merge(Relationship(sourceNode,relation, targetNode))





# create or megre a comptency node and set relationship between it and source node
def competencyProcess(sourceNode, competency, competencyValue, relationName, level=None):

    if (pd.isna(competencyValue) or len(competencyValue) < 1): return

    nodeName = competency # Node to process
    nodeAttr = competency[:1].lower() + competency[1:] # lower case for first character
    
    # separate competency to each single value
    for comp in competencyValue.split(','):
        if (len(comp) < 2): continue
        
        # make the property 
        property = {nodeAttr: comp.strip().upper()}

        # if level is not none, add level to properties
        if level is not None:
            level = level.upper()
            property["level"] = level

        compe = getOrCreateNodeMultiAttr(nodeName, nodeAttr, comp.strip().upper(), property, level)

        makeRelationship(sourceNode, relationName, compe)



def processAllCompetency(Knowledge, ProgrammingLanguage, Framework, Platform, Tool, level=None):

    # check if competency of this row is empty or not, if not then do nothing
    if (pd.isna(ProgrammingLanguage) or len(ProgrammingLanguage) < 1): return

    # separate competencies into single value
    for program in ProgrammingLanguage.split(','):
        if (len(program) < 2): continue
         # make the property 
        property = {"programmingLanguage": program.strip().upper()}

        # if level is not none, add level to properties
        if level is not None:
            level = level.upper()
            property["level"] = level
        Program = getOrCreateNodeMultiAttr("ProgrammingLanguage", "programmingLanguage", program.strip().upper(), property, level)
        competencyProcess(Program, "Knowledge", Knowledge, "Relate_to_knowledge")
        competencyProcess(Program, "Framework", Framework, "Have_framework", level)
        competencyProcess(Program, "Tool", Tool, "Use_tool") 

    if (pd.isna(Framework) or len(Framework) < 1): return

    for frame in Framework.split(','):
        if (len(frame) < 2): continue
        property = {"framework": frame.strip().upper()}

        # if level is not none, add level to properties
        if level is not None:
            property["level"] = level.upper()

        frameNode = getOrCreateNodeMultiAttr("Framework", "framework", frame.strip().upper(), property , level)
        competencyProcess(frameNode, "Platform", Platform, "Deploy_to_platform")
        competencyProcess(frameNode, "Knowledge", Knowledge, "Relate_to_framework")

In [12]:
def importJobData(df):
    # Iterate through rows and create nodes and relationships
    mileStone = len(df)
    i = 0
    for index, row in df.iterrows():
            
        Organization = getOrCreateNode("Organization", "name", row['companyName'])
        Location = getOrCreateNode( "Location", "location", row['location'])
      
        webName = row['website']
        webUrl =  "https://www." + webName + ".com"
        Website = getOrCreateNodeMultiAttr("Website", "name", webName, {"name": webName, "url": webUrl})

        # Create Career node or retrieve existing one
        Career = getOrCreateNode( "Career", "name", row['jobType'])
    
        # Create FactJobPosting node or retrieve existing one
        FactJobPosting = Node("FactJobPosting", totalJobPost=row['jobCount'])
        graph.create(FactJobPosting)

        # Create Date node or retrieve existing one
        timePost = getOrCreatetDateNode(row['date'],row['month'], row['year'])
        
        makeRelationship(FactJobPosting, "Recruited_by", Organization)
        makeRelationship(FactJobPosting, "Published_on", Website)
        makeRelationship(FactJobPosting, "Belong_to_career", Career)
        makeRelationship(FactJobPosting, "Located_at", Location)
        makeRelationship(FactJobPosting, "Posted_at_time", timePost)
        
        competencyProcess(FactJobPosting, "Knowledge", row['Knowledge'], "Required_knowledge" )
        competencyProcess(FactJobPosting, "ProgrammingLanguage", row['ProgrammingLanguage'], "Required_programmingLanguage" )
        competencyProcess(FactJobPosting, "Framework", row['Framework'], "Required_framework" )
        competencyProcess(FactJobPosting, "Platform", row['Platform'], "Required_platform" )
        competencyProcess(FactJobPosting, "Tool", row['Tool'], "Required_tool")

        i = i + 1
        print(f"Loading process {i/mileStone * 100} %", end='\r')
        


In [13]:
def divideFile(data, fileName, fileSize, fixSize=1200):
    lenght = len(data)
    size = int(lenght/fixSize * fileSize) # divide 1124 because default file size is 1,2 GB onl disk
    fileName = fileName + str(fileSize) + "MB.csv"
    dataToWrite = data.iloc[0:size]
    dataToWrite.to_csv(fileName, index=False)
    print("Done")

In [22]:
jobDataPath = "/Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/ETLCode/CleanData/job2024Clean.csv"
jobDataDf = pd.read_csv(jobDataPath)
jobDataDf = jobDataDf.fillna("")

divideFile(jobDataDf, DEFAULT_PATH + "jobDataSpecial", 1, 2)


Done


In [18]:
data = pd.read_csv(DEFAULT_PATH + "jobDataSpecial1MB.csv")
len(data)

538

In [19]:
importJobData(data)

In [23]:
importJobData(jobDataDf)